# Подготовка окружения

In [ ]:
!git clone https://github.com/your_username/dla_asr_hw1.git
%cd dla_asr_hw1
!pip install -r requirements.txt

# Загрузка весов

In [ ]:
!curl -Lo "saved/model_best.pth" "$(curl -sG --data-urlencode "public_key=https://yadi.sk/d/dT-0ta5WHuKO_Q" "https://cloud-api.yandex.net/v1/disk/public/resources/download" | jq -r .href)"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0:--:-- --:--:--     0
100  231M  100  231M    0     0   9.9M      0  0:00:23  0:00:23 --:--:-- 12.0M


# Загрузка пользовательского датасета

In [ ]:
import os

dataset_link = input("Введите ссылку на ваш датасет (Google Drive / public link): ")


!gdown --folder "$dataset_link" -O data/

# Находим самую новую папку, созданную в data/
downloaded_dirs = [d for d in os.listdir("data") if os.path.isdir(os.path.join("data", d))]
downloaded_dirs.sort(key=lambda x: os.path.getmtime(os.path.join("data", x)), reverse=True)
dataset_dir = os.path.join("data", downloaded_dirs[0]) if downloaded_dirs else None

print(f"📂 Найден загруженный датасет: {dataset_dir}")


# Запуск инференса

In [10]:
!python inference.py \
    inferencer.from_pretrained=saved/model_best.pth \
    datasets=custom \
    datasets.custom.audio_dir=data/custom/audio \
    datasets.custom.transcription_dir=data/custom/transcriptions \
    dataloader.batch_size=1

ConformerCTC(
  (encoder): ConformerEncoder(
    (sub): Conv2dSubsampling4(
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
      (out): Linear(in_features=5120, out_features=256, bias=True)
    )
    (layers): ModuleList(
      (0-11): 12 x ConformerBlock(
        (ff1): FeedForwardModule(
          (net): Sequential(
            (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=256, out_features=1024, bias=True)
            (2): Swish()
            (3): Dropout(p=0.1, inplace=False)
            (4): Linear(in_features=1024, out_features=256, bias=True)
            (5): Dropout(p=0.1, inplace=False)
          )
        )
        (mhsa): MHSA(
          (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (attn): Multi

# Табличка с результатами

In [11]:
import pandas as pd
pd.read_csv('inference_saved/custom/predictions.csv').head(10)

,file,target,prediction,wer,cer
0,26-496-0012.flac,and hardly anything of reformation though they...,and hardly anything of reformation though they...,13.8889,2.3923
